![paleoCAMP logo](./images/paleoCAMPLogo.png)

# CESM Output and Analysis


**Tutorial at [the 2024 paleoCAMP](https://paleoclimate.camp/) | June 18–July 1, 2024**  

       
[Jiang Zhu](https://staff.cgd.ucar.edu/jiangzhu/)  
[jiangzhu@ucar.edu](mailto:jiangzhu@ucar.edu)  
Climate & Global Dynamics Laboratory  
NSF National Center for Atmospheric Research  

------

**Learning Objectives:**

* Learn to use the NCAR JupyterHub for data access and analysis
* Learn to read and examine netcdf files using Xarray
* Learn techniques to make basic visualization of temperature, precipitation, and sea-surface temperature

**Time to learn**: 50 minutes

---
**How to get started?**  
1. Launch a NCAR JupyterHub server with `Casper PBS Batch`
   - Click `File` and then `Hub Control Panel`
   - `Add New Server` casper  
   ![JupyterHub Add Casper](./images/jupyterhub_add_casper.png)
   - Choose `Casper PBS Batch`  
   ![JupyterHub Casper Login](./images/jupyterhub_casper_login.png)
2. Launch a terminal within JupyterHub 
3. git clone https://github.com/jiang-zhu/paleocamp.git
4. Find and open `3_analyze_CESM_output.ipynb` from the left sidebar

---
Load Python packages

In [ ]:
import os
import glob
from datetime import timedelta

import xarray as xr
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point

# geocat is used for interpolation the atmosphere output
from geocat.comp import interpolation

# xesmf is used for regridding ocean output
import xesmf

import warnings
warnings.filterwarnings("ignore")

---
## Analysis 1: plot solar insolation in the MH to further validate the simulation

### Load data
***_Important:_*** use your own directories only if you have both piControl and midHolocene simulations finished
- Replace `YOUR_USERNAME` in the `ls` command with your username to see whether you have the required files

In [ ]:
!ls /glade/derecho/scratch/YOUR_USERNAME/archive/b.e21.B1850.f19_g17.piControl.001/atm/hist/*cam.h0.*
!ls /glade/derecho/scratch/YOUR_USERNAME/archive/b.e21.B1850.f19_g17.midHolocene.001/atm/hist/*cam.h0.*

In [ ]:
# replace YOUR_USERNAME with your username if you see 12-month piControl and midHolocene data
storage_dir = '/glade/derecho/scratch/YOUR_USERNAME/archive/'

# Uncoment the following if you don't have all the 24 files
# storage_dir = '/glade/derecho/scratch/jiangzhu/archive/'

- Use `glob` to obtain the file names including path.
- Use wildcard,`*`, to catch all files in the atmosphere history directory (see _Wildcard_ in _0_Prerequisites_1_unix.ipynb_).

In [ ]:
hist_dir = '/atm/hist/'

case_PI = 'b.e21.B1850.f19_g17.piControl.001'
case_MH = 'b.e21.B1850.f19_g17.midHolocene.001'

files_PI = glob.glob(storage_dir + case_PI + hist_dir + '*.cam.h0.0001*')
files_MH = glob.glob(storage_dir + case_MH + hist_dir + '*.cam.h0.0001*')
print(*files_PI, sep='\n')
print(*files_MH, sep='\n')

- Use `xr.open_mfdataset` to open multiple files at once in parallel (12 month in this case)
- We will keep using the Xarray Datasets `ds_MH` and `ds_PI` throughout the Notebook

In [ ]:
ds_PI = xr.open_mfdataset(files_PI)
ds_MH = xr.open_mfdataset(files_MH)

# We need this fix to get the correct time, i.e., January to December of year 1
ds_PI['time'] = ds_PI.time.get_index('time') - timedelta(days=15)
ds_MH['time'] = ds_MH.time.get_index('time') - timedelta(days=15)

In [ ]:
ds_PI

# Explore the dataset and find the Solar insolation, SOLIN

### Compute the zonal mean of solar insolation and make plots
- Use `.mean('lon')` to get the zonal mean
- Use the Xarray plotting functionality directly

In [ ]:
ds_PI.SOLIN

In [ ]:
# Compute zonal mean and make plot of piControl
ds_PI.SOLIN.mean('lon').plot.contourf(x='time', y='lat', figsize=(4, 2))

In [ ]:
# Add your code to compute zonal mean and make plot of midHolocene


In [ ]:
# Add your code to compute and plot the difference: midHolocen - piControl zonal mean


<div class="alert alert-success">   
<details>
 
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
Copy and paste the code into the above cell

```python

ds_MH.SOLIN.mean('lon').plot.contourf(x='time', y='lat', figsize=(4, 2))


(ds_MH.SOLIN - ds_PI.SOLIN).mean('lon').plot.contourf(
    x='time', y='lat', figsize=(4, 2), levels=np.linspace(-30, 30, 21))

```

</details>
</div>

### Small group discussion
- Which orbital parameters are different at the middle Holocene (6ka BP)?
- How does the orbital parameter impact the top-of-atmosphere shortwave radiation (solar insolation)
- Do the results look correct? You can compare your results with Figure 3b of [Otto-Bliesner et al., (2017)](https://gmd.copernicus.org/articles/10/3979/2017/gmd-10-3979-2017.pdf)

---
## Analysis 2: Does the Earth receive more radiation during the midHolocene?
- We compute the global annual mean solar insolation to answer this question.
- ***Importantly,*** we need to weight grid cells by their area (equivalent to cosine of latitude), using the `.weighted` method

Note that lat is in degree

In [ ]:
ds_PI.lat

### Example calculation for the piControl

In [ ]:
coslat = np.cos(np.deg2rad(ds_PI.lat))

SOLIN_PI = ds_PI.SOLIN.weighted(coslat).mean(('lat', 'lon', 'time'))
print("The global annual mean insolation of PI: ", SOLIN_PI.values, "W/m2")

### If we don't apply the area weights, we will get a wrong answer

In [ ]:
SOLIN_PI_unweighted = ds_PI.SOLIN.mean(('lat', 'lon', 'time'))
print("Only if we forgot to properly weight the values by area:",
      SOLIN_PI_unweighted.values, "W/m2")

### Add your own calculation for the mid-Holocene

<div class="alert alert-success">   
<details>
 
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
Copy and paste the code into the above cell

```python
coslat = np.cos(np.deg2rad(ds_MH.lat))

SOLIN_MH = ds_MH.SOLIN.weighted(coslat).mean(('lat', 'lon', 'time'))
print("The global annual mean insolation of MH: ", SOLIN_MH.values, "W/m2")
print("Difference, MH - PI: ", SOLIN_MH.values-SOLIN_PI.values, "W/m2")

```

</details>
</div>

### Small group discussion
- Should the mid-Holocene be warmer or colder than the preindustrial?
  - Geological records suggest that MH may be warmer than the present day
  - Climate models in general suggest a colder mid-Holocene
  - Further reading: [Osman et al. (2021)](https://www-nature-com.cuucar.idm.oclc.org/articles/s41586-021-03984-4)
  - Ask Jess and Jiang about the _Holocene Temperature Conundrum_

---
## Analysis 3: how does the midHolocene orbital forcing impact the ITCZ and monsoon precipitation?
- In CESM, total precipitation is computed as `prec = PRECC + PRECL` (sum of convective and large-scale precipitation; recall that climate models have to parameterize convection!)
- We convert the units from m/s into mm/day
- We use `.isel(time=slice(5, 8))` to select the June, July, and Agugust values (recall that Python uses 0-based ordering)
- Let's use Cartopy and Matplotlib with a Robinson projection (instead of using the simple Xarray.plot)
- We use `add_cyclic_point` to get rid of the "white strip" in the plot

In [ ]:
ds_PI.PRECC

In [ ]:
m_p_s_to_mm_p_day = 86400000

prec_PI = (ds_PI.PRECC + ds_PI.PRECL).isel(time=slice(5, 8)).mean('time') * m_p_s_to_mm_p_day
prec_PI

### Plot total precipitation of piControl

In [ ]:
fig, ax = plt.subplots(figsize=(3, 1.5), subplot_kw={
    'projection': ccrs.Robinson(central_longitude=210)})

p1 = ax.contourf(prec_PI.lon, prec_PI.lat, prec_PI,
                 cmap='YlGnBu',
                 levels=np.linspace(0, 17, 18),
                 extend='both',
                 transform=ccrs.PlateCarree())

ax.coastlines(linewidth=0.5)
plt.colorbar(p1)
ax.set_title("PI precipitation")

### Let's use `add_cyclic_point` to get rid of the "white strip" in the plot

In [ ]:
fig, ax = plt.subplots(figsize=(3, 1.5), subplot_kw={
    'projection': ccrs.Robinson(central_longitude=210)})

# Note the differences in the next two lines
prec_PI_new, lon_new = add_cyclic_point(prec_PI, prec_PI.lon)
p1 = ax.contourf(lon_new, prec_PI.lat, prec_PI_new,
                 cmap='YlGnBu',
                 levels=np.linspace(0, 17, 18),
                 extend='both',
                 transform=ccrs.PlateCarree())

ax.coastlines(linewidth=0.5)
plt.colorbar(p1)
ax.set_title("PI precipitation")

### Add your own plot of the midHolocene precipitatin

In [ ]:
fig, ax = plt.subplots(figsize=(3, 1.5), subplot_kw={
    'projection': ccrs.Robinson(central_longitude=210)})



### Add your own plot of the midHolocene - piControl

In [ ]:
fig, ax = plt.subplots(figsize=(3, 1.5), subplot_kw={
    'projection': ccrs.Robinson(central_longitude=210)})



### Discussion
- Do you see the ITCZ shift during the mid-Holocene?
- Do you see changes of the monsoon precipitation?
- Ask Kathleen, Tripti, and Kevin about ITCZ and monsoon!

---
## Analysis 4: how does the MH orbital forcing impact the atmospheric circulation?
- We plot the zonal mean zonal wind of PI in the Northern Hemisphere summer (JJA)
- We use `plt.gca().invert_yaxis()` to invert the y-axis such that the high pressure is at the bottom

In [ ]:
ds_PI.U

### Plot the zonal mean of u-winds of JJA

In [ ]:
U_PI_za_jja = ds_PI.U.isel(time=slice(5, 8)).mean(('lon', 'time'))

U_PI_za_jja.plot.contourf(figsize=(3, 1.5),
                          levels=np.linspace(-50, 50, 21), extend='both', )
plt.gca().invert_yaxis()

### Add your own plot to show the changes during the midHolocene

<div class="alert alert-success">   
<details>
 
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
Copy and paste the code into the above cell

```python
dU_MH_za_jja = (ds_MH.U - ds_PI.U).isel(time=slice(5, 8)).mean(('lon', 'time'))

dU_MH_za_jja.plot.contourf(figsize=(3, 1.5),
                          levels=np.linspace(-10, 10, 21), extend='both', )
plt.gca().invert_yaxis()
```

</details>
</div>

### Did you notice any problem with the above plots? 
- Check out the name of the vertical levels. it is called [hybrid sigma-pressure coordinate](https://www2.cesm.ucar.edu/models/atm-cam/docs/usersguide/node25.html), which is ***NOT*** the pressure coordinate.
- We need to interpolate from the hybrid sigma-pressure coordinate into the normal pressure coordinate

In [ ]:
ds_PI.lev.standard_name

### Use geocat interpolation to interpolate from hybrid sigma-pressure to presure coordinate

In [ ]:
P_new_mb = np.array([1, 10, 20, 50., 100., 200., 300.,
                    400., 500., 600., 700., 800., 900., 1000.])
p0_mb = 1000.

Up_PI = interpolation.interp_hybrid_to_pressure(
    ds_PI.U,
    ds_PI.PS,
    ds_PI.hyam,
    ds_PI.hybm,
    p0=p0_mb*100.,
    new_levels=P_new_mb*100.,
    extrapolate=False)

Up_MH = interpolation.interp_hybrid_to_pressure(
    ds_MH.U,
    ds_MH.PS,
    ds_MH.hyam,
    ds_MH.hybm,
    p0=p0_mb*100.,
    new_levels=P_new_mb*100.,
    extrapolate=False)

### Plot the correct results!

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2,
                         figsize=(8, 2),
                         constrained_layout=True)

Up_PI.isel(time=slice(5, 8)).mean(('lon', 'time')).plot.contourf(
    ax=axes[0], levels=np.linspace(-50, 50, 21), extend='both')

(Up_MH - Up_PI).isel(time=slice(5, 8)).mean(('lon', 'time')).plot.contourf(
    ax=axes[1], levels=np.linspace(-10, 10, 21), extend='both')

for ax in axes:
    ax.invert_yaxis()

### Small group discussion
- Which hemisphere has a stronger jet stream? Why? (remember that the plots are for JJA, the NH summer)
- Does the mid-Holocene orbital forcing shift the jet stream?
- Ask Tripti and Kevin about the atmosphere circulation! 

---
## Analysis 5: how about sea-surface temperature?
- Ocean data is in `ocn/hist`
- SST is the top level of TEMP

In [ ]:
hist_dir = '/ocn/hist/'

case_PI = 'b.e21.B1850.f19_g17.piControl.001'
case_MH = 'b.e21.B1850.f19_g17.midHolocene.001'

files_PI_ocn = glob.glob(storage_dir + case_PI + hist_dir + '*.pop.h.0001*')
files_MH_ocn = glob.glob(storage_dir + case_MH + hist_dir + '*.pop.h.0001*')
print(*files_PI_ocn, sep='\n')
print(*files_MH_ocn, sep='\n')

ds_PI_ocn = xr.open_mfdataset(files_PI_ocn)
ds_MH_ocn = xr.open_mfdataset(files_MH_ocn)

# Again, we need this fix to get the correct time, i.e., month 1 to month 12
ds_PI_ocn['time'] = ds_PI_ocn.time.get_index('time') - timedelta(days=15)
ds_MH_ocn['time'] = ds_MH_ocn.time.get_index('time') - timedelta(days=15)

In [ ]:
ds_PI_ocn

### Calculate annual mean SST and make a simple plot using Xarray

In [ ]:
sst_PI = ds_PI_ocn.TEMP.isel(z_t=0).mean('time')
sst_MH = ds_MH_ocn.TEMP.isel(z_t=0).mean('time')

sst_PI.plot(size=1.5)

- What do nlat and nlon mean?
- Where is Greenland? It is the "north pole" in the model!  
![POP ocean grid](./images/pop_gx1_grid.png)
<p style="text-align: center;"> Figure: POP ocean grid</p>

### Regridding is needed!

In [ ]:
%%time

ds_PI_ocn['lat'] = ds_PI_ocn.TLAT
ds_PI_ocn['lon'] = ds_PI_ocn.TLONG

regridder = xesmf.Regridder(
    ds_in=ds_PI_ocn,
    ds_out=xesmf.util.grid_global(1, 1, cf=True, lon1=360),
    method='bilinear',
    periodic=True)

sst_PI_1x1 = regridder(sst_PI)
sst_MH_1x1 = regridder(sst_MH)
sst_PI_1x1

In [ ]:
lat = sst_PI_1x1.lat
lon = sst_PI_1x1.lon

fig, axes = plt.subplots(nrows=1, ncols=3,
                         figsize=(10, 2),
                         subplot_kw={'projection': ccrs.Robinson(central_longitude=210)},
                         constrained_layout=True)

ax = axes[0]
ax.set_title("PI SST, annual mean")
sst_PI_new, lon_new = add_cyclic_point(sst_PI_1x1, lon)
p0 = ax.contourf(lon_new, lat, sst_PI_new,
                 levels=np.linspace(-2, 30, 17),
                 cmap='inferno', extend='both',
                 transform=ccrs.PlateCarree())
plt.colorbar(p0, ax=ax)

ax = axes[1]
ax.set_title("MH SST, annual mean")
sst_MH_new, lon_new = add_cyclic_point(sst_MH_1x1, lon)
p1 = ax.contourf(lon_new, lat, sst_MH_new,
                 levels=np.linspace(-2, 30, 17),
                 cmap='inferno', extend='both',
                 transform=ccrs.PlateCarree())
plt.colorbar(p1, ax=ax)

ax = axes[2]
ax.set_title("MH-PI SST")
p2 = ax.contourf(lon_new, lat, sst_MH_new - sst_PI_new,
                 cmap='coolwarm',
                 levels=np.linspace(-2, 2, 21),
                 extend='both',
                 transform=ccrs.PlateCarree())
plt.colorbar(p2, ax=ax)

for ax in axes:
    ax.set_global()
    ax.coastlines(linewidth=0.5)

# We could use savefig to save the plot as pdf
# plt.savefig('SST_xy.PI_vs_MH.pdf', format='pdf', bbox_inches="tight")

### Small group discussion
- Colder SSTs over lots of the regions? Internal variability?
- We need to test significance
- What's the timescale of upper ocean response?

---
## Summary so far
- Load multiple CESM output using `Xarray`
- Plot variables in zonal mean and map view
- Interpolate the atmosphere vertical coordinate from hybrid to pressure is needed!
- Ocean models usually use irregular grid with poles moved into land; regridding is needed!
- Mid-Holocene orbital forcing is seasonal and impacts atmosphere circulation, precipitation, and other climate states.